# RAG 기본 버전

이 노트북은 RAG 시스템의 기본적인 기능만 포함합니다:
- PDF 본문/부칙 분리 및 청킹
- CSV 표 데이터 처리
- FAISS 벡터 스토어 생성
- 기본 RAG 질의응답


In [44]:
# 환경 설정
import os
import sys
from pathlib import Path
from typing import List, Optional

import pandas as pd
import re

# Upstage API Key 설정
os.environ.setdefault("UPSTAGE_API_KEY", "up_EoF0I0CzeHxuDYmf0we544GMPCFIT")
UPSTAGE_API_KEY = os.environ.get("UPSTAGE_API_KEY", "")

# 작업 디렉토리 설정
CURRENT_DIR = Path(__file__).parent if "__file__" in globals() else Path.cwd()
if Path.cwd() != CURRENT_DIR:
    os.chdir(CURRENT_DIR)

print(f"✅ 작업 디렉토리: {CURRENT_DIR}")
print(f"✅ Upstage API Key: {UPSTAGE_API_KEY[:20]}...")


✅ 작업 디렉토리: c:\Users\janen\Documents\25-2 강의\자연어처리\프로젝트\ewha
✅ Upstage API Key: up_EoF0I0CzeHxuDYmf0...


In [45]:
# 필요한 라이브러리 설치
%pip install langchain_upstage langchain_community langchain-openai openai pdfplumber pandas faiss-cpu wikipedia-api wikipedia


Note: you may need to restart the kernel to use updated packages.


In [46]:
# ==================================================================
# 1. PDF 본문/부칙 분리 및 청킹 (다현님 작업 기반)
# ==================================================================

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

PDF_PATH = CURRENT_DIR / "ewha.pdf"

if not PDF_PATH.exists():
    raise FileNotFoundError(f"{PDF_PATH} 파일을 찾을 수 없습니다.")

print("📄 PDF 파일 로딩 중...")
loader = PyPDFLoader(str(PDF_PATH))
docs = loader.load()
docs = [d for d in docs if int(d.metadata["page"]) < 38]  # 부칙 이전 페이지만
print(f"✅ 총 {len(docs)}페이지 로드 완료")

# 전체 텍스트 병합
full_text = "\n".join([d.page_content for d in docs])

# "부칙" 등장 지점 찾기
split_index = full_text.find("부칙")
print(f"📌 부칙 시작 인덱스: {split_index}")

# 본문 / 부칙 분리
if split_index != -1:
    main_text = full_text[:split_index]
    appendix_text = full_text[split_index:]
else:
    main_text = full_text
    appendix_text = ""

print(f"📄 본문 길이: {len(main_text)}")
print(f"📄 부칙 길이: {len(appendix_text)}")

# ==================================================================
# PDF 구조 분석 및 조항별 청킹 함수
# ==================================================================

def split_by_articles(text: str, is_appendix: bool = False) -> List[str]:
    """
    PDF 텍스트를 조항별로 정확히 분리하는 함수
    이화여대 학칙 구조 분석:
    - 별표 1, 별표 2, 별표 3 등 (표)
    - 제1조, 제2조, 제3조 등 (조항)
    - ①, ②, ③ 등 (항목)
    - 부칙 (부칙 시작)
    """
    import re
    
    # 조항 시작 패턴 (우선순위 순서대로)
    # 1. 별표 (표는 독립적인 단위)
    # 2. 제X조 (조항)
    # 3. ①항, ②항 등 (항목 - 조항 내 세부사항)
    
    splits = []
    
    # 패턴 1: 별표로 시작하는 부분 (가장 큰 단위)
    star_pattern = r'(별표\s*\d+)'
    star_matches = list(re.finditer(star_pattern, text))
    
    # 패턴 2: 제X조로 시작하는 부분
    article_pattern = r'(제\d+조(?:\s*\([^)]+\))?)'  # 제1조, 제1조(목적) 등
    article_matches = list(re.finditer(article_pattern, text))
    
    # 별표가 있으면 별표 기준으로 먼저 분리
    if star_matches:
        last_pos = 0
        for match in star_matches:
            pos = match.start()
            if pos > last_pos:
                chunk = text[last_pos:pos].strip()
                if chunk:
                    splits.append(chunk)
            last_pos = pos
        # 마지막 별표 이후 텍스트
        if last_pos < len(text):
            chunk = text[last_pos:].strip()
            if chunk:
                splits.append(chunk)
        
        # 별표로 분리된 각 청크를 제X조로 다시 분리
        refined_splits = []
        for split in splits:
            split_article_matches = list(re.finditer(article_pattern, split))
            if split_article_matches:
                split_last_pos = 0
                for match in split_article_matches:
                    pos = match.start()
                    if pos > split_last_pos:
                        chunk = split[split_last_pos:pos].strip()
                        if chunk:
                            refined_splits.append(chunk)
                    split_last_pos = pos
                if split_last_pos < len(split):
                    chunk = split[split_last_pos:].strip()
                    if chunk:
                        refined_splits.append(chunk)
            else:
                refined_splits.append(split)
        splits = refined_splits
    elif article_matches:
        # 별표가 없으면 제X조 기준으로 분리
        last_pos = 0
        for match in article_matches:
            pos = match.start()
            if pos > last_pos:
                chunk = text[last_pos:pos].strip()
                if chunk:
                    splits.append(chunk)
            last_pos = pos
        # 마지막 조항 이후 텍스트
        if last_pos < len(text):
            chunk = text[last_pos:].strip()
            if chunk:
                splits.append(chunk)
    else:
        # 별표도 제X조도 없으면 전체를 하나의 청크로
        splits = [text]
    
    # 패턴 3: ①항, ②항 등으로 시작하는 부분 (긴 조항 내에서만)
    # 이는 나중에 세부 청킹에서 처리
    
    # 부칙의 경우 "부칙" 키워드로도 분리
    if is_appendix and splits:
        final_splits = []
        for split in splits:
            # "부칙" 키워드가 중간에 나오면 분리
            if "부칙" in split:
                parts = re.split(r'(부칙)', split, maxsplit=1)
                if len(parts) > 1:
                    if parts[0].strip():
                        final_splits.append(parts[0].strip())
                    if len(parts) > 2:
                        final_splits.append(parts[1] + parts[2].strip())
                else:
                    final_splits.append(split)
            else:
                final_splits.append(split)
        splits = final_splits
    
    # 빈 청크 제거 및 최소 길이 체크
    splits = [s for s in splits if s and len(s.strip()) > 50]
    
    return splits if splits else [text]

# 본문을 조항별로 분리
print("📋 본문 조항별 분리 중...")
main_articles = split_by_articles(main_text, is_appendix=False)
print(f"   → {len(main_articles)}개 조항으로 분리됨")

# 각 조항을 청킹 (조항이 너무 길 경우만)
main_chunks = []
for i, article in enumerate(main_articles):
    # 조항이 1200자 이하면 그대로 사용
    if len(article) <= 1200:
        main_chunks.append(Document(
            page_content=article,
            metadata={"source": "ewha.pdf 본문", "type": "main_text", "article_index": i}
        ))
    else:
        # 조항이 길면 세부 항목별로 분리
        text_splitter_main = RecursiveCharacterTextSplitter(
            chunk_size=1200,
            chunk_overlap=200,  # overlap 감소 (조항 경계 유지)
            separators=["\n\n①", "\n\n②", "\n\n③", "\n\n④", "\n\n⑤", "\n\n⑥", "\n\n⑦", "\n\n⑧", "\n\n⑨", "\n\n⑩",
                       "\n①", "\n②", "\n③", "\n④", "\n⑤", "\n⑥", "\n⑦", "\n⑧", "\n⑨", "\n⑩",
                       "\n\n", "\n", ". ", " "],
        )
        sub_chunks = text_splitter_main.split_text(article)
        for j, chunk in enumerate(sub_chunks):
            main_chunks.append(Document(
                page_content=chunk,
                metadata={"source": "ewha.pdf 본문", "type": "main_text", "article_index": i, "sub_index": j}
            ))

print(f"✅ 본문 청크 수: {len(main_chunks)}개")

# 부칙을 조항별로 분리
if appendix_text:
    print("📋 부칙 조항별 분리 중...")
    appendix_articles = split_by_articles(appendix_text, is_appendix=True)
    print(f"   → {len(appendix_articles)}개 조항으로 분리됨")
    
    appendix_chunks = []
    for i, article in enumerate(appendix_articles):
        # 부칙 조항은 더 작게 청킹 (세부 규정이 많음)
        if len(article) <= 1000:
            appendix_chunks.append(Document(
                page_content=article,
                metadata={"source": "ewha.pdf 부칙", "type": "appendix_text", "article_index": i}
            ))
        else:
            # 부칙 조항이 길면 세부 항목별로 분리
            text_splitter_appendix = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=150,  # overlap 감소
                separators=["\n\n①", "\n\n②", "\n\n③", "\n\n④", "\n\n⑤", "\n\n⑥", "\n\n⑦", "\n\n⑧", "\n\n⑨", "\n\n⑩",
                           "\n①", "\n②", "\n③", "\n④", "\n⑤", "\n⑥", "\n⑦", "\n⑧", "\n⑨", "\n⑩",
                           "별표", "\n\n", "\n", ". ", " "],
            )
            sub_chunks = text_splitter_appendix.split_text(article)
            for j, chunk in enumerate(sub_chunks):
                appendix_chunks.append(Document(
                    page_content=chunk,
                    metadata={"source": "ewha.pdf 부칙", "type": "appendix_text", "article_index": i, "sub_index": j}
                ))
    
    print(f"✅ 부칙 청크 수: {len(appendix_chunks)}개")
else:
    appendix_chunks = []

print(f"\n📦 총 텍스트 청크: {len(main_chunks) + len(appendix_chunks)}개")
print(f"   - 본문: {len(main_chunks)}개")
print(f"   - 부칙: {len(appendix_chunks)}개")


📄 PDF 파일 로딩 중...
✅ 총 38페이지 로드 완료
📌 부칙 시작 인덱스: 20919
📄 본문 길이: 20919
📄 부칙 길이: 20222
📋 본문 조항별 분리 중...
   → 108개 조항으로 분리됨
✅ 본문 청크 수: 108개
📋 부칙 조항별 분리 중...
   → 108개 조항으로 분리됨
✅ 부칙 청크 수: 109개

📦 총 텍스트 청크: 217개
   - 본문: 108개
   - 부칙: 109개


In [47]:
# ==================================================================
# 2. CSV 표 데이터 처리 및 문장 생성 (다현님 + 사용자 작업 통합)
# ==================================================================

def coalesce(*values: object) -> str:
    """첫 번째 유효한 값을 반환"""
    for value in values:
        if value is None:
            continue
        if isinstance(value, float) and pd.isna(value):
            continue
        text = str(value).strip()
        if text:
            return text
    return ""

# 다현님 작업: 학위 문장 생성 방식
def build_degree_sentences_from_csv(df: pd.DataFrame, year: str = "최신 개정") -> List[str]:
    """CSV에서 학위 문장 생성 (다현님 방식)"""
    sentences = []
    for idx, row in df.iterrows():
        college = coalesce(row.get("설치대학"), row.get("대학"))
        degree = coalesce(row.get("학위_종류"), row.get("학위종류"), row.get("학사"))
        major = coalesce(row.get("학과_전공"), row.get("학과 또는 전공"), row.get("전공"))
        
        if not college or not degree or not major:
            continue
            
        sentence = f"{college}의 {major} 전공은 {degree} 학위를 수여한다. ({year})"
        sentences.append(sentence)
    return sentences

# 사용자 작업: 표 데이터 처리 함수들
def build_quota_text(row: pd.Series) -> str:
    """입학정원 텍스트 생성 (capacity.csv 구조에 맞게 수정)"""
    year = coalesce(row.get("year"), row.get("학년도"))
    college = coalesce(row.get("college"), row.get("대학"))
    department = coalesce(row.get("department"), row.get("학부"))
    major = coalesce(row.get("major"), row.get("학과_전공"), row.get("학과 또는 전공"))
    quota = coalesce(row.get("quota"), row.get("정원"), row.get("입학정원_명"))
    
    # quota 값 정리: "[53]" 같은 괄호 부분 제거
    if quota and not pd.isna(quota):
        import re
        quota_str = str(quota).strip()
        # 괄호와 그 안의 내용 제거 (예: "99[53]" -> "99")
        quota_str = re.sub(r'\[.*?\]|\(.*?\)', '', quota_str).strip()
        quota = quota_str
    
    # 빈 값 체크
    if not quota or pd.isna(quota) or str(quota).strip() == "":
        quota = "미상"
    
    # 연도 포맷팅
    if year and not pd.isna(year):
        year_str = f"{int(year)}학년도" if str(year).isdigit() else str(year)
    else:
        year_str = ""
    
    # 문장 구성
    parts = []
    if year_str:
        parts.append(year_str)
    if college:
        parts.append(college)
    if department:
        parts.append(department)
    
    prefix = " ".join(parts) if parts else ""
    
    # 전공이 있으면 전공 포함
    if major and str(major).strip():
        return f"{prefix} 소속 {major}의 입학 정원은 {quota}명입니다."
    # 전공이 없고 department에 학과명이 있으면 department 사용
    elif department and str(department).strip() and ("과" in str(department) or "학부" in str(department)):
        return f"{prefix} 소속 {department}의 입학 정원은 {quota}명입니다."
    # department만 있으면
    elif department:
        return f"{prefix} 소속 {department}의 입학 정원은 {quota}명입니다."
    # college만 있으면
    elif college:
        return f"{prefix}의 입학 정원은 {quota}명입니다."
    else:
        return f"{year_str} 입학 정원은 {quota}명입니다."

def build_contract_text(row: pd.Series) -> str:
    """계약학과 텍스트 생성 (사용자 방식)"""
    college = coalesce(row.get("설치대학"))
    form = coalesce(row.get("설치형태"))
    major = coalesce(row.get("학과_전공"))
    degree = coalesce(row.get("학위_종류"))
    quota = coalesce(row.get("입학정원_명"))
    period = coalesce(row.get("설치_운영기간"))
    
    parts = [f"계약학과 설치·운영 정보: 설치대학={college}"]
    if form:
        parts.append(f"설치형태={form}")
    if major:
        parts.append(f"학과/전공={major}")
    if degree:
        parts.append(f"수여 학위={degree}")
    if quota:
        parts.append(f"입학 정원={quota}명")
    if period:
        parts.append(f"설치·운영 기간={period}")
    return ", ".join(parts) + "."

def build_grade_text(row: pd.Series) -> str:
    """성적점 텍스트 생성 (grade.csv 구조에 맞게)"""
    year = coalesce(row.get("year"))
    grade = coalesce(row.get("grade"))
    gpa = coalesce(row.get("gpa"))
    
    if not year or not grade or pd.isna(gpa):
        return ""
    
    # year가 "from 1980"이면 "1980학년도 이후", "before 1980"이면 "1980학년도 이전"
    if "from 1980" in str(year).lower() or "1980 이후" in str(year):
        year_desc = "1980학년도 이후 입학생"
    elif "before 1980" in str(year).lower() or "1980 이전" in str(year):
        year_desc = "1980학년도 이전 입학생"
    else:
        year_desc = str(year)
    
    return f"{year_desc}에 적용하는 등급 {grade}의 성적점은 {gpa}점입니다."

# CSV 파일 처리 (ewha csvs 폴더 통합)
csv_documents = []

# CSV 파일 경로 설정 (ewha csvs 폴더 우선)
csvs_dir = CURRENT_DIR / "ewha csvs"
if not csvs_dir.exists():
    csvs_dir = CURRENT_DIR  # 폴더가 없으면 현재 디렉토리 사용

# 1) degrees 관련 파일 처리 (ewha csvs 폴더 우선)
degrees_dir = csvs_dir / "degrees" if (csvs_dir / "degrees").exists() else CURRENT_DIR / "degrees"
degree_sentences_file = csvs_dir / "degree_sentences_all.csv의 사본" if (csvs_dir / "degree_sentences_all.csv의 사본").exists() else degrees_dir / "degree_sentences_all.csv의 사본"
degree_latest_file = csvs_dir / "degree_latest.csv의 사본" if (csvs_dir / "degree_latest.csv의 사본").exists() else degrees_dir / "degree_latest.csv의 사본"

# 우선순위 1: 이미 문장으로 변환된 파일 사용
if degree_sentences_file.exists():
    df_sentences = pd.read_csv(degree_sentences_file, encoding='utf-8-sig')
    for _, row in df_sentences.iterrows():
        sentence = str(row.get("sentence", "")).strip()
        if sentence:
            csv_documents.append(Document(
                page_content=sentence,
                metadata={"source": "[별표 2] 학사학위의 종류", "page": 51, "type": "degree"}
            ))
    print(f"✅ degree_sentences_all.csv 처리 완료: {len(df_sentences)}개 문장")
# 우선순위 2: degree_latest.csv를 사용하여 문장 생성
elif degree_latest_file.exists():
    df_degrees = pd.read_csv(degree_latest_file, encoding='utf-8-sig')
    # 빈 행 제거
    df_degrees = df_degrees.dropna(subset=['대학', '학사', '전공'], how='all')
    # 다현님 방식: 학위 문장 생성
    degree_sentences = build_degree_sentences_from_csv(df_degrees, "최신 개정")
    for sentence in degree_sentences:
        csv_documents.append(Document(
            page_content=sentence,
            metadata={"source": "[별표 2] 학사학위의 종류", "page": 51, "type": "degree"}
        ))
    print(f"✅ degree_latest.csv 처리 완료: {len(degree_sentences)}개 문장")
# 우선순위 3: 기존 degrees.csv 사용 (하위 호환성)
elif (CURRENT_DIR / "degrees.csv").exists():
    df_degrees = pd.read_csv(CURRENT_DIR / "degrees.csv", encoding='utf-8-sig')
    degree_sentences = build_degree_sentences_from_csv(df_degrees, "최신 개정")
    for sentence in degree_sentences:
        csv_documents.append(Document(
            page_content=sentence,
            metadata={"source": "[별표 2] 학사학위의 종류", "page": 51, "type": "degree"}
        ))
    print(f"✅ degrees.csv 처리 완료: {len(degree_sentences)}개 문장")
else:
    print("⚠️ degrees 관련 파일을 찾을 수 없습니다.")

# 2) capacity.csv 처리 (입학정원 데이터)
capacity_path = csvs_dir / "capacity.csv" if (csvs_dir / "capacity.csv").exists() else CURRENT_DIR / "capacity.csv"
if capacity_path.exists():
    df_capacity = pd.read_csv(capacity_path, encoding='utf-8-sig')
    # 빈 행 제거
    df_capacity = df_capacity.dropna(subset=['year', 'college', 'quota'], how='all')
    
    # quota 값 정리: "[53]" 같은 괄호 부분 제거 (예: "99[53]" -> "99")
    def clean_quota(quota_val):
        if pd.isna(quota_val):
            return quota_val
        quota_str = str(quota_val).strip()
        # 괄호와 그 안의 내용 제거 (예: "99[53]" -> "99", "100(50)" -> "100")
        import re
        quota_str = re.sub(r'\[.*?\]|\(.*?\)', '', quota_str).strip()
        return quota_str
    
    df_capacity['quota'] = df_capacity['quota'].apply(clean_quota)
    
    # "전체" 행 제거 (department나 major에 "전체"가 포함된 경우)
    # 단, "의과대학 전체" 같은 경우는 quota 값이 유효하면 유지 (대학 전체 정원 정보)
    # quota 값이 이미 정리되었으므로 숫자만 있는지 확인
    df_capacity = df_capacity[
        ~(
            # department와 major 모두에 "전체"가 있는 경우만 제거
            (df_capacity['department'].astype(str).str.contains('전체', na=False)) &
            (df_capacity['major'].astype(str).str.contains('전체', na=False))
        ) |
        (
            # "전체" 행이지만 quota 값이 유효한 숫자인 경우는 유지 (대학 전체 정원)
            (df_capacity['department'].astype(str).str.contains('전체', na=False)) &
            (df_capacity['quota'].astype(str).str.match(r'^\d+$', na=False))  # 숫자만 있는 경우
        )
    ]
    # major가 있거나, department에 학과명이 있는 행만 사용
    # (major가 비어있어도 department에 구체적인 학과명이 있으면 포함)
    df_capacity = df_capacity[
        (
            (df_capacity['major'].notna()) & 
            (df_capacity['major'] != '') &
            (df_capacity['major'].astype(str).str.strip() != '')
        ) |
        (
            (df_capacity['department'].notna()) & 
            (df_capacity['department'] != '') &
            (df_capacity['department'].astype(str).str.strip() != '') &
            (df_capacity['department'].astype(str).str.contains('과|부|학부', na=False))
        )
    ]
    
    for _, row in df_capacity.iterrows():
        text = build_quota_text(row)
        if text and text.strip():
            year = str(row.get("year", "")).strip()
            csv_documents.append(Document(
                page_content=text,
                metadata={
                    "source": f"[별표 1] {year}학년도 입학정원" if year else "[별표 1] 입학정원",
                    "page": 39,
                    "type": "quota",
                    "year": year
                }
            ))
    print(f"✅ capacity.csv 처리 완료: {len(df_capacity)}개 문서")
else:
    print("⚠️ capacity.csv 파일이 없습니다.")

# 3) contract_dept.csv 처리
contract_path = csvs_dir / "contract_dept.csv" if (csvs_dir / "contract_dept.csv").exists() else CURRENT_DIR / "contract_dept.csv"
if contract_path.exists():
    df_contract = pd.read_csv(contract_path, encoding='utf-8-sig')
    for _, row in df_contract.iterrows():
        text = build_contract_text(row)
        csv_documents.append(Document(
            page_content=text,
            metadata={"source": "[별표 3] 계약학과 설치·운영", "page": 53, "type": "contract"}
        ))
    print(f"✅ contract_dept.csv 처리 완료: {len(df_contract)}개 문서")
else:
    print("⚠️ contract_dept.csv 파일이 없습니다.")

# 4) grade.csv 처리 (1980년 기준 성적점 데이터)
grade_path = csvs_dir / "grade.csv" if (csvs_dir / "grade.csv").exists() else CURRENT_DIR / "grade.csv"
if grade_path.exists():
    df_grade = pd.read_csv(grade_path, encoding='utf-8-sig')
    for _, row in df_grade.iterrows():
        text = build_grade_text(row)
        if text and text.strip():
            csv_documents.append(Document(
                page_content=text,
                metadata={"source": "[별표] 성적점 등급표", "page": 0, "type": "grade"}
            ))
    print(f"✅ grade.csv 처리 완료: {len(df_grade)}개 문서")
else:
    print("⚠️ grade.csv 파일이 없습니다.")

print(f"\n📦 총 CSV 문서: {len(csv_documents)}개")


✅ degree_sentences_all.csv 처리 완료: 160개 문장
✅ capacity.csv 처리 완료: 582개 문서
✅ contract_dept.csv 처리 완료: 1개 문서
✅ grade.csv 처리 완료: 26개 문서

📦 총 CSV 문서: 769개


In [ ]:
# ==================================================================
# 3. 벡터 스토어 생성 및 임베딩
# ==================================================================

from langchain_upstage import UpstageEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import wikipedia

# 임베딩 모델 로드
embeddings = UpstageEmbeddings(
    api_key=UPSTAGE_API_KEY,
    model="solar-embedding-1-large"
)
print("✅ Upstage 임베딩 모델 로드 완료")

# 모든 문서 통합
all_documents = []
all_documents.extend(main_chunks)
if appendix_chunks:
    all_documents.extend(appendix_chunks)
if csv_documents:
    all_documents.extend(csv_documents)

print(f"📚 총 {len(all_documents)}개 문서 청크 생성")

# ==================================================================
# 3-1. 수동 지식 문서 벡터화 (선택)
# ==================================================================
manual_documents = []
manual_dir = CURRENT_DIR / "manual_rules"
if manual_dir.exists():
    print("\n📘 수동 지식 문서 로딩 중...")
    for file in manual_dir.glob("*.txt"):
        try:
            text = file.read_text(encoding="utf-8").strip()
        except Exception as e:
            print(f"   ⚠️ {file.name} 읽기 실패: {e}")
            continue
        if not text:
            continue
        text_splitter_manual = RecursiveCharacterTextSplitter(
            chunk_size=800,
            chunk_overlap=200,
            separators=["\n\n", "\n", ". ", " "]
        )
        chunks = text_splitter_manual.split_text(text)
        for i, chunk in enumerate(chunks):
            manual_documents.append(Document(
                page_content=chunk,
                metadata={
                    "source": f"manual:{file.name}",
                    "type": "manual",
                    "priority": "high",
                    "chunk_index": i
                }
            ))
    if manual_documents:
        all_documents.extend(manual_documents)
        print(f"   ✅ 수동 지식 청크 {len(manual_documents)}개 추가")
else:
    print("📘 수동 지식 문서가 없습니다.")

print(f"📚 총 {len(all_documents)}개 문서 청크 생성 (수동 지식 포함)")

# ==================================================================
# 3-2. Wikipedia 문서 벡터화 (최우선 문서)
# ==================================================================

def fetch_wikipedia_article(title: str, language: str = "en") -> Optional[str]:
    """Wikipedia 문서 가져오기"""
    try:
        wikipedia.set_lang(language)
        page = wikipedia.page(title, auto_suggest=False)
        return page.content
    except wikipedia.exceptions.DisambiguationError as e:
        # 동음이의어인 경우 첫 번째 옵션 사용
        try:
            page = wikipedia.page(e.options[0], auto_suggest=False)
            return page.content
        except:
            return None
    except Exception as e:
        print(f"⚠️ Wikipedia 문서 '{title}' 가져오기 실패: {e}")
        return None

def create_wikipedia_document(title: str, content: str, priority: str = "high") -> List[Document]:
    """Wikipedia 문서를 청크로 분할"""
    if not content:
        return []
    
    # 긴 문서는 청킹
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", ". ", " "]
    )
    
    chunks = text_splitter.split_text(content)
    documents = []
    for i, chunk in enumerate(chunks):
        documents.append(Document(
            page_content=chunk,
            metadata={
                "source": f"Wikipedia: {title}",
                "type": "wikipedia",
                "priority": priority,
                "chunk_index": i
            }
        ))
    
    return documents

# 최우선 Wikipedia 문서 목록 (Phase 1)
priority_1_wiki_titles = {
    "Kohlberg's stages of moral development": "high",
    "Robbery": "high",
    "Larceny": "high",
    "Burglary": "high",
    "Homo erectus": "high",
    "Neoteny": "high",
}

# 차순위 Wikipedia 문서 목록 (Phase 2)
priority_2_wiki_titles = {
    "Tang dynasty": "medium",
    "Deutschland über alles": "medium",
    "Bipolar disorder": "medium",
    "Ethics in psychology": "medium",
    "Utilitarianism": "medium",
    "Categorical imperative": "medium",
    "Jurisprudence": "medium",
    "Famine, Affluence and Morality": "medium",
}

# 보조 Wikipedia 문서 목록 (Phase 3)
priority_3_wiki_titles = {
    "Human evolution": "low",
    "Homo habilis": "low",
    "Australopithecus afarensis": "low",
    "Aristotelianism": "low",
    "Peter Singer": "low",
    "Green marketing": "low",
    "Theory of multiple intelligences": "low",
    "Approach-approach conflict": "low",
    "Receiving stolen property": "low",
    "Legal positivism": "low",
    "John Frere": "low",  # Q46 추가
    "Du Fu": "low",  # Q29 추가
    "Recognition of foreign judgments": "low",  # Q42 추가
}

# 벡터화할 문서 선택 (기본값: 전체 적용)
WIKI_PHASE = 3  # 1: 최우선, 2: 최우선+차순위, 3: 전체

wiki_documents = []

if WIKI_PHASE >= 1:
    print("\n📖 Phase 1: 최우선 Wikipedia 문서 벡터화 중...")
    for title, priority in priority_1_wiki_titles.items():
        print(f"   - {title} 다운로드 중...")
        content = fetch_wikipedia_article(title)
        if content:
            docs = create_wikipedia_document(title, content, priority)
            wiki_documents.extend(docs)
            print(f"     ✅ {len(docs)}개 청크 생성")
        else:
            print(f"     ❌ 다운로드 실패")

if WIKI_PHASE >= 2:
    print("\n📖 Phase 2: 차순위 Wikipedia 문서 벡터화 중...")
    for title, priority in priority_2_wiki_titles.items():
        print(f"   - {title} 다운로드 중...")
        content = fetch_wikipedia_article(title)
        if content:
            docs = create_wikipedia_document(title, content, priority)
            wiki_documents.extend(docs)
            print(f"     ✅ {len(docs)}개 청크 생성")
        else:
            print(f"     ❌ 다운로드 실패")

if WIKI_PHASE >= 3:
    print("\n📖 Phase 3: 보조 Wikipedia 문서 벡터화 중...")
    for title, priority in priority_3_wiki_titles.items():
        print(f"   - {title} 다운로드 중...")
        content = fetch_wikipedia_article(title)
        if content:
            docs = create_wikipedia_document(title, content, priority)
            wiki_documents.extend(docs)
            print(f"     ✅ {len(docs)}개 청크 생성")
        else:
            print(f"     ❌ 다운로드 실패")

if wiki_documents:
    all_documents.extend(wiki_documents)
    print(f"\n✅ Wikipedia 문서 {len(wiki_documents)}개 청크 추가 완료")

print(f"\n📚 총 {len(all_documents)}개 문서 청크 생성 (학칙: {len(all_documents) - len(wiki_documents)}, Wikipedia: {len(wiki_documents)})")

# FAISS 벡터 스토어 생성
vector_store = FAISS.from_documents(all_documents, embeddings)
print("✅ FAISS 벡터 스토어 생성 완료")


✅ Upstage 임베딩 모델 로드 완료
📚 총 986개 문서 청크 생성

📖 Phase 1: 최우선 Wikipedia 문서 벡터화 중...
   - Kohlberg's stages of moral development 다운로드 중...
     ✅ 45개 청크 생성
   - Robbery 다운로드 중...
     ✅ 25개 청크 생성
   - Larceny 다운로드 중...
     ✅ 47개 청크 생성
   - Burglary 다운로드 중...
     ✅ 36개 청크 생성
   - Homo erectus 다운로드 중...
     ✅ 46개 청크 생성
   - Neoteny 다운로드 중...
     ✅ 20개 청크 생성

📖 Phase 2: 차순위 Wikipedia 문서 벡터화 중...
   - Tang dynasty 다운로드 중...
     ✅ 154개 청크 생성
   - Deutschland über alles 다운로드 중...
     ✅ 34개 청크 생성
   - Bipolar disorder 다운로드 중...
     ✅ 109개 청크 생성
   - Ethics in psychology 다운로드 중...
     ✅ 191개 청크 생성
   - Utilitarianism 다운로드 중...
     ✅ 129개 청크 생성
   - Categorical imperative 다운로드 중...
     ✅ 51개 청크 생성
   - Jurisprudence 다운로드 중...
     ✅ 66개 청크 생성

📖 Phase 3: 보조 Wikipedia 문서 벡터화 중...
   - Human evolution 다운로드 중...
     ✅ 135개 청크 생성
   - Homo habilis 다운로드 중...
     ✅ 46개 청크 생성
   - Australopithecus afarensis 다운로드 중...
     ✅ 57개 청크 생성
   - Aristotelianism 다운로드 중...
     ✅ 27개 청크 생성
   - Peter Sin

In [ ]:
# ==================================================================
# 4. 기본 RAG 질의응답
# ==================================================================

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage

# LLM 모델 로드
llm = ChatUpstage(api_key=UPSTAGE_API_KEY, model="solar-pro2")

# 질문 분류 프롬프트 템플릿
classification_prompt_template = PromptTemplate.from_template(
    """
    다음 질문이 이화여자대학교 학칙(입학, 학사, 성적, 학위, 전공, 수강, 졸업 등) 관련 문제인지 판단하세요.
    
    이화여대 학칙 관련 문제의 예시:
    - 이화여대의 입학정원, 학위 종류, 학과 정보
    - 이화여대의 학사 규정, 성적 점수, 학점 이수
    - 이화여대의 전공, 복수전공, 부전공 관련 규정
    - 이화여대의 졸업 요건, 재학 연한 관련 규정
    
    이화여대 학칙과 무관한 문제의 예시:
    - 일반적인 역사, 과학, 심리학, 철학 문제
    - 다른 대학이나 일반적인 법률 문제
    - Wikipedia에서 찾을 수 있는 일반 지식 문제
    
    질문: {question}
    
    응답 형식: "YES" 또는 "NO"만 답변하세요.
    - "YES": 이화여대 학칙 관련 문제
    - "NO": 이화여대 학칙과 무관한 문제
    """
)

# RAG 프롬프트 템플릿 (개선 버전 - 범용적 정확도 향상)
rag_prompt_template = PromptTemplate.from_template(
    """
    다음 질문에 대해 주어진 컨텍스트를 바탕으로 가장 정확하게 답변하세요.
    
    **중요 지침:**
    1. 이화여대 학칙 문서를 우선 참고하세요 (구체적인 규정)
    2. Wikipedia 문서는 일반적인 학문 지식 참고용입니다
    3. 컨텍스트에서 정확한 조항을 인용하여 답변하세요
    4. **각 선택지를 개별적으로 검증하세요** - 모든 선택지를 순서대로 확인하고 정확한 것을 선택하세요
    5. 정확히 하나의 선택지만 답변하세요 (예: "정답: (A)" 또는 "정답: (A)" 또는 "(A)")
    6. 여러 선택지가 언급되어도 최종 답변은 하나만 제시하세요
    7. 컨텍스트에 정보가 없으면 정확히 "정보가 없습니다"라고 답변하세요
    
    **답변 형식 (반드시 준수):**
    - 마지막에 "정답: (X)" 형식으로 명확히 표시하세요
    - 예: "정답: (A)" 또는 "정답은 (A)입니다"
    - 설명이 필요하면 간단히 추가하되, 선택지는 반드시 명확히 표시
    
    **특별 주의사항:**
    - "해당하지 않는 것", "옳지 않은 것", "잘못된 것" 등 부정형 질문에 주의하세요
      → 질문이 무엇을 찾는지 정확히 파악하고, 각 선택지를 컨텍스트와 대조하세요
    - 수학 계산 문제는 단계별로 계산하여 정확한 답을 구하세요
    - 법률 용어는 정확한 정의를 확인하세요 (robbery vs larceny vs burglary 등)
      → 각 용어의 정확한 정의와 차이점을 확인하세요
    - 심리학 이론의 단계는 정확한 번호를 확인하세요 (Stage 1, Stage 2 등)
      → 단계 번호와 특징을 정확히 매칭하세요
    - 역사/문학 문제는 맥락을 고려하세요
      → 작품의 역사적 배경과 작가의 의도를 함께 고려하세요
    
    **답변 절차:**
    1. 질문의 핵심을 파악하세요 (무엇을 묻는지, 부정형인지)
    2. 각 선택지를 컨텍스트와 대조하세요
    3. 선택지별 판단을 "(A) O/X - 이유" 형식으로 간결히 기록하세요
    4. 가장 정확한 선택지를 선택하고 마지막에 "정답: (X)" 형식으로 명확히 표시하세요
    
    **출력 템플릿 (예시)**
    선택지 검토:
    - (A) O/X - 근거
    - (B) ...
    결론: ...
    정답: (X)
    
    ---
    질문: {question}
    ---
    컨텍스트:
    {context}
    """
)

# 일반 질문 프롬프트 템플릿 (RAG 없이)
general_prompt_template = PromptTemplate.from_template(
    """
    다음 질문에 대해 답변하세요.
    ---
    질문: {question}
    """
)

# 체인 구성
classification_chain = classification_prompt_template | llm
rag_chain = rag_prompt_template | llm
general_chain = general_prompt_template | llm

# Retriever 설정 (개선: 검색 개수 증가)
# 이화여대 학칙 관련 문제는 더 많은 문서를 검색하여 정확도 향상
retriever = vector_store.as_retriever(
    search_kwargs={
        "k": 10  # 검색 개수 증가 (5 → 10) - 범용적 정확도 향상
    }
)

# MMR (Maximum Marginal Relevance) Retriever 설정
# 다양성을 고려한 검색으로 중복된 문서 제거
try:
    from langchain_community.retrievers import BM25Retriever
    from langchain.retrievers import EnsembleRetriever
    # BM25 + FAISS 하이브리드 검색 (키워드 + 벡터)
    # BM25는 키워드 기반 검색으로 정확도 향상
    print("✅ 하이브리드 검색 준비 완료")
except ImportError:
    print("⚠️ BM25Retriever를 사용할 수 없습니다. 기본 검색만 사용합니다.")

print("✅ RAG 체인 구성 완료")

# 질문 분류 함수
def classify_question(question: str) -> bool:
    """질문이 이화여대 학칙 관련 문제인지 판단"""
    try:
        response = classification_chain.invoke({"question": question})
        answer = response.content.strip().upper()
        # "YES" 또는 "YES"로 시작하는 경우 True
        return answer.startswith("YES")
    except Exception as e:
        print(f"⚠️ 질문 분류 중 오류: {e}")
        # 오류 발생 시 기본값: 이화여대 관련 키워드가 있으면 True
        ewha_keywords = ["이화", "이화여대", "이화여자대학교", "ewha"]
        question_lower = question.lower()
        return any(keyword in question_lower for keyword in ewha_keywords)

# 질문 예시
def ask_question(question: str):
    """질문에 대한 답변 생성 (자동 분류)"""
    # 질문 분류
    is_ewha_related = classify_question(question)
    
    if is_ewha_related:
        # 이화여대 학칙 관련 문제: RAG 사용
        source_docs = retriever.invoke(question)
        context_text = "\n\n".join(doc.page_content for doc in source_docs) or ""
        response = rag_chain.invoke({"question": question, "context": context_text})
        print(f"\n[이화여대 학칙 관련] 질문: {question}")
        print(f"답변: {response.content}")
        print(f"참고 문서: {len(source_docs)}개")
    else:
        # 일반 문제: RAG 없이 LLM만 사용
        response = general_chain.invoke({"question": question})
        print(f"\n[일반 문제] 질문: {question}")
        print(f"답변: {response.content}")
        print(f"참고 문서: 없음 (RAG 미사용)")
    
    return response

# 예시 질문
# ask_question("이화여대의 입학정원은 어떻게 되나요?")


✅ 하이브리드 검색 준비 완료
✅ RAG 체인 구성 완료


In [50]:
# ==================================================================
# 5. testset.csv에서 질문 불러와서 RAG 기반 답변 생성 및 채점
# ==================================================================

# testset.csv 파일 경로 설정 (프로젝트 루트 디렉토리)
TESTSET_PATH = CURRENT_DIR.parent / "testset.csv"

# testset.csv 파일 읽기 함수
def read_testset(file_path: Path):
    """testset.csv 파일에서 질문과 정답 읽기"""
    try:
        data = pd.read_csv(file_path, encoding='utf-8-sig')
        prompts = data["prompts"].tolist() if "prompts" in data.columns else []
        answers = data["answers"].tolist() if "answers" in data.columns else None
        return prompts, answers
    except Exception as e:
        print(f"❌ 파일 읽기 오류: {e}")
        return [], None

# 답변 추출 함수 (개선 버전 - 범용적 정확도 향상)
def extract_answer(response: str, question: Optional[str] = None) -> Optional[str]:
    """
    응답에서 답변 추출 (개선 버전 - 범용적)
    - 여러 선택지가 있을 경우 가장 확실한 것 선택
    - "정답:", "Answer:", "답:" 등의 키워드 우선
    - 응답의 마지막 부분에서 최종 답변 추출
    - 유효한 선택지만 추출 (A-Z만 허용)
    - "정답: **C**" 같은 마크다운 형식도 처리
    """
    import re
    
    if not response or not response.strip():
        return None
    
    # 유효한 선택지 (일반적으로 사용되는 선택지)
    valid_choices = set('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    
    # 응답을 문장 단위로 분리하여 마지막 부분 우선 검색
    sentences = response.split('.')
    last_sentences = sentences[-5:] if len(sentences) > 5 else sentences  # 마지막 5문장
    
    # 우선순위 1: 명시적인 정답 표시 (마지막 문장에서 우선 검색)
    # "정답: **C**", "정답: (C)", "정답은 (C)입니다" 등 다양한 형식 지원
    patterns = [
        r"정답[은는]?\s*[:\s]*\*{0,2}\s*\(([A-Z])\)",  # 정답: **C** 또는 정답: (C)
        r"정답[은는]?\s*[:\s]*\*{1,2}([A-Z])\*{0,2}",  # 정답: **C** 또는 정답: C
        r"정답[:\s]*\(([A-Z])\)",
        r"Answer[:\s]*\(([A-Z])\)",
        r"답[:\s]*\(([A-Z])\)",
        r"\[ANSWER\]:\s*\(([A-Z])\)",
        r"The answer is\s*\(([A-Z])\)",
        r"Correct answer[:\s]*\(([A-Z])\)",
        r"최종[:\s]*답[:\s]*\(([A-Z])\)",
        r"정답[은는]?\s*[:\s]*([A-Z])\b",  # 정답: C (괄호 없음)
    ]
    
    # 마지막 문장부터 역순으로 검색
    for sentence in reversed(last_sentences):
        for pattern in patterns:
            match = re.search(pattern, sentence, re.IGNORECASE)
            if match:
                answer = match.group(1).upper()
                if answer in valid_choices:
                    return answer
    
    # 전체 응답에서도 검색
    for pattern in patterns:
        match = re.search(pattern, response, re.IGNORECASE)
        if match:
            answer = match.group(1).upper()
            if answer in valid_choices:
                return answer
    
    # 우선순위 2: "정답은 (X)입니다" 형식
    pattern_final = r"정답은\s*\(([A-Z])\)\s*입니다"
    match = re.search(pattern_final, response, re.IGNORECASE)
    if match:
        answer = match.group(1).upper()
        if answer in valid_choices:
            return answer
    
    # 우선순위 3: (X) 형식 (마지막 발생 - 보통 최종 답변)
    # 마지막 문장에서 우선 검색
    for sentence in reversed(last_sentences):
        pattern2 = r"\(([A-Z])\)"
        matches = list(re.finditer(pattern2, sentence))
        if matches:
            answer = matches[-1].group(1).upper()
            if answer in valid_choices:
                return answer
    
    # 전체 응답에서 검색
    pattern2 = r"\(([A-Z])\)"
    matches = list(re.finditer(pattern2, response))
    if matches:
        # 여러 개가 있으면 마지막 것 (보통 최종 답변)
        answer = matches[-1].group(1).upper()
        if answer in valid_choices:
            return answer
    
    # 우선순위 4: 단독 문자 (마지막 발생, 문장 끝 근처)
    # 마지막 문장에서 우선 검색
    for sentence in reversed(last_sentences):
        # 문장 끝에 있는 단독 문자 우선 (예: "정답: **C**")
        pattern3 = r"([A-Z])\s*[\.\*]*\s*$"  # 문장 끝의 단독 문자
        match = re.search(pattern3, sentence.strip())
        if match:
            answer = match.group(1).upper()
            if answer in valid_choices:
                return answer
        
        # 일반 단독 문자
        pattern3 = r"\b([A-Z])\b"
        matches = list(re.finditer(pattern3, sentence))
        if matches:
            for match in reversed(matches):
                answer = match.group(1).upper()
                if answer in valid_choices:
                    return answer
    
    # 전체 응답에서 검색
    pattern3 = r"\b([A-Z])\b"
    matches = list(re.finditer(pattern3, response))
    if matches:
        # 마지막 단독 문자 (보통 최종 답변)
        for match in reversed(matches):
            answer = match.group(1).upper()
            if answer in valid_choices:
                return answer
    
    return None

# 정답 추출 함수 (정답 문자열에서 선택지 추출)
def extract_gold_answer(gold_str: str) -> Optional[str]:
    """정답 문자열에서 선택지 문자 추출"""
    import re
    if pd.isna(gold_str) or not gold_str:
        return None
    
    gold_str = str(gold_str).strip()
    # (A), (B) 형식 추출
    match = re.search(r"\(([A-Z])\)", gold_str)
    if match:
        return match.group(1).upper()
    
    # 단독 문자 추출
    match = re.search(r"\b([A-Z])\b", gold_str)
    if match:
        return match.group(1).upper()
    
    return None

# testset.csv 파일 로드
print(f"📂 testset.csv 파일 경로: {TESTSET_PATH}")
prompts, gold_answers = read_testset(TESTSET_PATH)

if not prompts:
    print(f"❌ {TESTSET_PATH} 파일을 찾을 수 없거나 질문이 없습니다.")
    print("   파일 경로를 확인하세요.")
else:
    print(f"✅ 테스트 프롬프트 {len(prompts)}개 로드 완료")
    if gold_answers:
        print(f"✅ 정답 {len(gold_answers)}개 로드 완료")

# 키워드 추출 함수 (범용적)
def extract_keywords_from_question(question: str) -> List[str]:
    """질문에서 검색 키워드 추출 (범용적)"""
    import re
    keywords = []
    
    # 질문에서 주요 명사/키워드 추출
    # 영어 단어 추출
    english_words = re.findall(r'\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*\b', question)
    keywords.extend([w.lower() for w in english_words if len(w) > 3])
    
    # 한국어 키워드 추출 (2글자 이상)
    korean_words = re.findall(r'[가-힣]{2,}', question)
    keywords.extend(korean_words)
    
    # 선택지에서 키워드 추출
    choice_pattern = r'\([A-Z]\)\s*([^\(\)]+?)(?=\s*\([A-Z]\)|$)'
    choices = re.findall(choice_pattern, question)
    for choice in choices:
        # 선택지에서 중요한 단어 추출
        words = re.findall(r'\b[A-Z][a-z]+\b|[가-힣]{2,}', choice)
        keywords.extend([w.lower() if w.isalpha() else w for w in words if len(w) > 2])
    
    return list(set(keywords))[:10]  # 중복 제거 및 최대 10개

# 재검색 함수 (범용적 - 선택지 기반 재검색 포함)
def enhanced_retrieval(question: str, retriever, is_ewha_related: bool, max_docs: int = 10):
    """향상된 검색 (재검색 로직 포함 - 범용적)"""
    # 첫 번째 검색
    source_docs = retriever.invoke(question)
    existing_sources = {doc.metadata.get('source', '') + str(doc.metadata.get('article_index', '')) for doc in source_docs}
    
    # 검색 결과가 부족하거나 관련성이 낮으면 재검색
    if len(source_docs) < 5 or (is_ewha_related and len(source_docs) < 3):
        # 키워드 기반 재검색
        keywords = extract_keywords_from_question(question)
        if keywords:
            # 가장 중요한 키워드로 재검색
            for keyword in keywords[:3]:  # 상위 3개 키워드만 사용
                additional_docs = retriever.invoke(keyword)
                for doc in additional_docs:
                    doc_id = doc.metadata.get('source', '') + str(doc.metadata.get('article_index', ''))
                    if doc_id not in existing_sources:
                        source_docs.append(doc)
                        existing_sources.add(doc_id)
                if len(source_docs) >= max_docs:
                    break
        
        # 선택지 기반 재검색 (특히 학칙 관련 문제)
        if is_ewha_related:
            import re
            # 특정 키워드로 재검색 (복수전공, 신청 자격 등)
            special_keywords = []
            if "복수전공" in question or "신청 자격" in question:
                special_keywords = ["복수전공", "신청", "자격", "조건"]
            elif "졸업" in question:
                special_keywords = ["졸업", "요건", "기준"]
            elif "재입학" in question:
                special_keywords = ["재입학", "횟수", "제한"]
            
            for keyword in special_keywords:
                additional_docs = retriever.invoke(keyword)
                for doc in additional_docs:
                    doc_id = doc.metadata.get('source', '') + str(doc.metadata.get('article_index', ''))
                    if doc_id not in existing_sources:
                        source_docs.append(doc)
                        existing_sources.add(doc_id)
                if len(source_docs) >= max_docs:
                    break
            
            # 선택지에서 중요한 키워드 추출
            choice_pattern = r'\([A-Z]\)\s*([^\(\)]+?)(?=\s*\([A-Z]\)|$)'
            choices = re.findall(choice_pattern, question)
            for choice in choices[:2]:  # 상위 2개 선택지만
                # 선택지의 핵심 키워드 추출
                choice_keywords = re.findall(r'[가-힣]{2,}|\b[A-Z][a-z]+\b', choice)
                for keyword in choice_keywords[:2]:  # 각 선택지에서 2개 키워드
                    if len(keyword) > 2:
                        additional_docs = retriever.invoke(keyword)
                        for doc in additional_docs:
                            doc_id = doc.metadata.get('source', '') + str(doc.metadata.get('article_index', ''))
                            if doc_id not in existing_sources:
                                source_docs.append(doc)
                                existing_sources.add(doc_id)
                        if len(source_docs) >= max_docs:
                            break
    
    return source_docs[:max_docs]

# RAG 응답 생성 함수 (질문 분류 포함, 개선 버전 - 범용적)
def generate_rag_response(question: str, retriever, rag_chain, general_chain, classification_chain):
    """단일 질문에 대한 응답 생성 (자동 분류 후 RAG 또는 일반 LLM 사용, 개선 버전 - 범용적)"""
    # 질문 분류
    is_ewha_related = classify_question(question)
    
    # 수학 계산 문제 감지
    is_math_problem = any(keyword in question.lower() for keyword in [
        'invested', 'income', 'divided', 'calculate', 'how much', '얼마', '계산', '나눔', '투자'
    ])
    
    if is_ewha_related:
        # 이화여대 학칙 관련 문제: RAG 사용
        # 향상된 검색 (재검색 로직 포함)
        source_docs = enhanced_retrieval(question, retriever, is_ewha_related=True, max_docs=10)
        
        # 문서 관련성 필터링 (유사도 점수가 있는 경우)
        filtered_docs = []
        for doc in source_docs:
            # 메타데이터에 score가 있으면 확인
            if hasattr(doc, 'metadata') and 'score' in doc.metadata:
                if doc.metadata['score'] >= 0.2:  # 임계값 낮춤 (더 많은 문서 포함)
                    filtered_docs.append(doc)
            else:
                filtered_docs.append(doc)
        
        # 필터링된 문서가 없으면 원본 사용
        if not filtered_docs:
            filtered_docs = source_docs[:8]  # 최대 8개 사용
        
        # 컨텍스트 생성 (문서 번호 추가로 참조 용이)
        context_parts = []
        for i, doc in enumerate(filtered_docs[:8], 1):  # 최대 8개 사용
            doc_text = doc.page_content
            source_info = doc.metadata.get('source', 'Unknown')
            context_parts.append(f"[문서 {i}] (출처: {source_info})\n{doc_text}")
        
        context_text = "\n\n".join(context_parts) or ""
        
        response = rag_chain.invoke({"question": question, "context": context_text})
        
        return {
            "response": response.content,
            "sources": [doc.metadata for doc in filtered_docs],
            "num_sources": len(filtered_docs),
            "is_ewha_related": True
        }
    else:
        # 일반 문제: Wikipedia 벡터화된 문서 사용
        # 향상된 검색 (재검색 로직 포함)
        source_docs = enhanced_retrieval(question, retriever, is_ewha_related=False, max_docs=10)
        
        # Wikipedia 문서 필터링 (priority가 있는 경우 우선)
        filtered_wiki_docs = []
        ewha_docs = []
        
        for doc in source_docs:
            doc_type = doc.metadata.get('type', '')
            if doc_type == 'wikipedia':
                filtered_wiki_docs.append(doc)
            elif doc_type in ['main_text', 'appendix_text', 'degree', 'quota', 'contract', 'grade']:
                ewha_docs.append(doc)
        
        # Wikipedia 문서가 있으면 사용
        if filtered_wiki_docs:
            # priority 순으로 정렬 (high > medium > low)
            priority_order = {'high': 3, 'medium': 2, 'low': 1}
            filtered_wiki_docs.sort(
                key=lambda x: priority_order.get(x.metadata.get('priority', 'low'), 1),
                reverse=True
            )
            
            # Wikipedia 문서 우선 사용
            context_parts = []
            for i, doc in enumerate(filtered_wiki_docs[:8], 1):  # 최대 8개 사용
                doc_text = doc.page_content
                source_info = doc.metadata.get('source', 'Unknown')
                priority = doc.metadata.get('priority', 'medium')
                context_parts.append(f"[문서 {i}] (출처: {source_info}, 우선순위: {priority})\n{doc_text}")
            
            context_text = "\n\n".join(context_parts) or ""
            response = rag_chain.invoke({"question": question, "context": context_text})
            
            return {
                "response": response.content,
                "sources": [doc.metadata for doc in filtered_wiki_docs],
                "num_sources": len(filtered_wiki_docs),
                "is_ewha_related": False
            }
        elif is_math_problem:
            # 수학 문제는 LLM에게 직접 계산 요청
            math_prompt = f"""
다음 수학 문제를 단계별로 정확히 계산하여 답변하세요.
각 단계를 명확히 보여주고, 마지막에 "정답: (X)" 형식으로 답변하세요.

문제: {question}
"""
            response = general_chain.invoke({"question": math_prompt})
            
            return {
                "response": response.content,
                "sources": [],
                "num_sources": 0,
                "is_ewha_related": False
            }
        else:
            # Wikipedia 문서가 없으면 LLM만 사용
            response = general_chain.invoke({"question": question})
            
            return {
                "response": response.content,
                "sources": [],
                "num_sources": 0,
                "is_ewha_related": False
            }

# 모든 질문에 대해 RAG 응답 생성
responses = []
if prompts and len(prompts) > 0:
    print(f"\n▶ 질문 분류 및 응답 생성 시작... (총 {len(prompts)}개 질문)")
    
    for idx, prompt in enumerate(prompts, 1):
        try:
            result = generate_rag_response(prompt, retriever, rag_chain, general_chain, classification_chain)
            responses.append({
                "question_num": idx,
                "question": prompt,
                "response": result["response"],
                "sources": result["sources"],
                "num_sources": result["num_sources"],
                "is_ewha_related": result.get("is_ewha_related", False)
            })
            
            # 진행 상황 출력 (10개마다 또는 마지막)
            if idx % 10 == 0 or idx == len(prompts):
                ewha_status = "이화여대 학칙 관련" if result.get("is_ewha_related", False) else "일반 문제"
                print(f"   진행: {idx}/{len(prompts)} 완료 ({ewha_status})")
                print(f"   최근 질문: {prompt[:60]}...")
                print(f"   최근 응답: {result['response'][:80]}...")
        except Exception as e:
            print(f"❌ 질문 {idx} 처리 중 오류: {e}")
            responses.append({
                "question_num": idx,
                "question": prompt,
                "response": f"오류 발생: {str(e)}",
                "sources": [],
                "num_sources": 0,
                "is_ewha_related": False
            })
    
    print(f"\n✅ RAG 응답 생성 완료: {len(responses)}개")
    
    # 결과를 DataFrame으로 변환
    results_df = pd.DataFrame(responses)
    
    # 정답이 있으면 추가 및 채점
    if gold_answers and len(gold_answers) == len(responses):
        results_df["gold_answer"] = gold_answers
        
        # 예측 답변 추출
        print("\n📝 답변 추출 중...")
        results_df["predicted_answer"] = results_df["response"].apply(
            lambda x: extract_answer(x) if pd.notna(x) else None
        )
        
        # 정답 추출
        results_df["gold_answer_extracted"] = results_df["gold_answer"].apply(
            lambda x: extract_gold_answer(x) if pd.notna(x) else None
        )
        
        # 정오 채점
        results_df["is_correct"] = (
            results_df["predicted_answer"] == results_df["gold_answer_extracted"]
        )
        
        # 정답률 계산
        total_questions = len(results_df)
        correct_count = results_df["is_correct"].sum()
        accuracy = (correct_count / total_questions * 100) if total_questions > 0 else 0
        
        print(f"✅ 답변 추출 및 채점 완료")
        print(f"\n📊 채점 결과:")
        print(f"   - 총 문제 수: {total_questions}개")
        print(f"   - 정답 수: {correct_count}개")
        print(f"   - 오답 수: {total_questions - correct_count}개")
        print(f"   - 정답률: {accuracy:.2f}%")
        
        # 질문 분류 통계
        if "is_ewha_related" in results_df.columns:
            ewha_related_count = results_df["is_ewha_related"].sum()
            general_count = total_questions - ewha_related_count
            print(f"\n📋 질문 분류 통계:")
            print(f"   - 이화여대 학칙 관련: {ewha_related_count}개")
            print(f"   - 일반 문제: {general_count}개")
        
        # 문제별 상세 정보 출력 (오답만) + 원인 분석
        wrong_answers = results_df[~results_df["is_correct"]]
        if len(wrong_answers) > 0:
            print(f"\n❌ 오답 문제 ({len(wrong_answers)}개):")
            wrong_nums = wrong_answers["question_num"].tolist() if "question_num" in wrong_answers.columns else []
            print(f"   문제 번호: {', '.join(map(str, wrong_nums))}")
            
            print("\n" + "=" * 80)
            print("오답 문제 상세 분석:")
            print("=" * 80)
            
            # 오답 원인 분석 함수
            def analyze_wrong_answer(row):
                """오답 원인 분석"""
                q_num = row.get("question_num", "N/A")
                pred = row["predicted_answer"] if pd.notna(row["predicted_answer"]) else None
                gold = row["gold_answer_extracted"] if pd.notna(row["gold_answer_extracted"]) else None
                response_text = row.get("response", "") if pd.notna(row.get("response", "")) else ""
                is_ewha = row.get("is_ewha_related", False) if pd.notna(row.get("is_ewha_related", False)) else False
                num_sources = row.get("num_sources", 0) if pd.notna(row.get("num_sources", 0)) else 0
                
                reasons = []
                
                # 원인 1: 답변 추출 실패
                if pred is None:
                    reasons.append("❌ 답변 추출 실패 - 응답에서 선택지를 추출하지 못함")
                    if "정보가 없습니다" in response_text or "not present" in response_text.lower():
                        reasons.append("   → 컨텍스트에 정보가 없음")
                    else:
                        reasons.append("   → 응답 형식이 예상과 다름")
                
                # 원인 2: 일반 문제인데 RAG 미사용
                elif not is_ewha and num_sources == 0:
                    reasons.append("⚠️ 일반 문제 - Wikipedia/외부 문서 벡터화 필요")
                    # 주제 추정
                    question_lower = str(row.get("question", "")).lower()
                    if any(kw in question_lower for kw in ["kohlberg", "moral", "stage"]):
                        reasons.append("   → 필요한 문서: Kohlberg's stages of moral development")
                    elif any(kw in question_lower for kw in ["robbery", "larceny", "burglary", "stolen"]):
                        reasons.append("   → 필요한 문서: Robbery, Larceny, Burglary 비교 문서")
                    elif any(kw in question_lower for kw in ["homo", "erectus", "hominid", "neoteny"]):
                        reasons.append("   → 필요한 문서: Homo erectus, Neoteny")
                    elif any(kw in question_lower for kw in ["tang", "dynasty", "nomadic"]):
                        reasons.append("   → 필요한 문서: Tang dynasty (nomadic peoples)")
                    elif any(kw in question_lower for kw in ["deutschland", "clemenceau"]):
                        reasons.append("   → 필요한 문서: Deutschland über alles, Clemenceau")
                
                # 원인 3: 학칙 관련 문제인데 잘못된 답변
                elif is_ewha:
                    reasons.append("⚠️ 이화여대 학칙 관련 문제 - 검색된 문서가 부정확하거나 부족함")
                    if num_sources < 3:
                        reasons.append(f"   → 검색된 문서 수 부족 ({num_sources}개)")
                
                # 원인 4: 답변은 추출했지만 잘못됨
                elif pred != gold:
                    reasons.append(f"❌ 잘못된 답변 추출 - 예측: {pred}, 정답: {gold}")
                    if not is_ewha:
                        reasons.append("   → 외부 문서 벡터화 필요")
                
                return reasons
            
            for _, row in wrong_answers.iterrows():
                q_num = row.get("question_num", "N/A")
                pred = row["predicted_answer"] if pd.notna(row["predicted_answer"]) else "추출 실패"
                gold = row["gold_answer_extracted"] if pd.notna(row["gold_answer_extracted"]) else "추출 실패"
                question_text = row.get("question", "") if pd.notna(row.get("question", "")) else ""
                response_text = row.get("response", "") if pd.notna(row.get("response", "")) else ""
                is_ewha = row.get("is_ewha_related", False) if pd.notna(row.get("is_ewha_related", False)) else False
                question_type = "이화여대 학칙 관련" if is_ewha else "일반 문제"
                
                print(f"\n[문제 {q_num}] ({question_type})")
                print(f"질문: {question_text}")
                print(f"예측 답변: {pred}")
                print(f"정답: {gold}")
                
                # 원인 분석
                reasons = analyze_wrong_answer(row)
                if reasons:
                    print("원인 분석:")
                    for reason in reasons:
                        print(f"  {reason}")
                
                print(f"생성된 응답: {response_text[:300]}..." if len(response_text) > 300 else f"생성된 응답: {response_text}")
                print("-" * 80)
        
        # 정답 문제 전체 출력
        correct_answers = results_df[results_df["is_correct"]]
        if len(correct_answers) > 0:
            print(f"\n✅ 정답 문제 ({len(correct_answers)}개):")
            correct_nums = correct_answers["question_num"].tolist() if "question_num" in correct_answers.columns else []
            print(f"   문제 번호: {', '.join(map(str, correct_nums))}")
            
            # 정답 문제 상세 정보 출력
            print("\n" + "=" * 80)
            print("정답 문제 상세:")
            print("=" * 80)
            for _, row in correct_answers.iterrows():
                q_num = row.get("question_num", "N/A")
                pred = row["predicted_answer"] if pd.notna(row["predicted_answer"]) else "추출 실패"
                gold = row["gold_answer_extracted"] if pd.notna(row["gold_answer_extracted"]) else "추출 실패"
                question_text = row.get("question", "") if pd.notna(row.get("question", "")) else ""
                is_ewha = row.get("is_ewha_related", False) if pd.notna(row.get("is_ewha_related", False)) else False
                question_type = "이화여대 학칙 관련" if is_ewha else "일반 문제"
                
                print(f"\n[문제 {q_num}] ({question_type}) - 정답: {gold}")
                print(f"질문: {question_text[:150]}..." if len(question_text) > 150 else f"질문: {question_text}")
                print("-" * 80)
        
        # 최종 점수 정보 추가
        results_df["final_score"] = f"{correct_count}/{total_questions} ({accuracy:.2f}%)"
        
    else:
        print("⚠️ 정답 정보가 없어 채점을 수행할 수 없습니다.")
    
    # 결과 저장
    output_path = CURRENT_DIR / "rag_responses.csv"
    results_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 응답을 {output_path}에 저장했습니다.")
    
    # 요약 정보 출력
    print(f"\n📊 요약:")
    print(f"   - 총 질문 수: {len(responses)}")
    print(f"   - 평균 참고 문서 수: {results_df['num_sources'].mean():.1f}")
    if "is_correct" in results_df.columns:
        print(f"   - 최종 점수: {results_df['final_score'].iloc[0] if 'final_score' in results_df.columns else 'N/A'}")
else:
    print("⚠️ 처리할 질문이 없습니다.")


📂 testset.csv 파일 경로: c:\Users\janen\Documents\25-2 강의\자연어처리\프로젝트\testset.csv
✅ 테스트 프롬프트 50개 로드 완료
✅ 정답 50개 로드 완료

▶ 질문 분류 및 응답 생성 시작... (총 50개 질문)
   진행: 10/50 완료 (일반 문제)
   최근 질문: QUESTION10) 이화여자대학교의 위치는 어디인가요?  (A) 강남구  (B) 서대문구  (C) 종로구 ...
   최근 응답: 이화여자대학교의 위치는 **서대문구**에 있습니다.  

정답: **(B) 서대문구**  

이화여자대학교의 정확한 주소는 **서울특별시 서대문...
   진행: 20/50 완료 (이화여대 학칙 관련)
   최근 질문: QUESTION20) 재학 연한 초과로 제적당하지 않는 경우는?
(A) 학사 편입
(B) 복수 전공 중
(C...
   최근 응답: 재학 연한 초과로 제적당하지 않는 경우를 찾기 위해 주어진 학칙 문서를 분석하면 다음과 같습니다:

1. **(A) 학사 편입**: 문서1에 따...
   진행: 30/50 완료 (일반 문제)
   최근 질문: QUESTION30)Homo erectus differed from Homo habilis in which ...
   최근 응답: **핵심 파악:**  
질문은 *Homo erectus*와 *Homo habilis*의 차이점을 묻고 있습니다. 선택지 중 정확한 차이를 찾아야...
   진행: 40/50 완료 (일반 문제)
   최근 질문: QUESTION40)____________ refers to a strategic process involv...
   최근 응답: 주어진 질문은 "____________ refers to a strategic process involving stakeholder assess...
   진행: 50/50 완료 (일반 문제)
   최근 질문: QUESTION50) Which is the least accurate descri